In [1]:
import sys
sys.path.append("../") # go to parent dir
# imports
import simpy
from loguru import logger
import time
import numpy as np
import pandas as pd
from cluster_simulator.utils import convex_hull
from cluster_simulator.cluster import Cluster, Tier, EphemeralTier, bandwidth_share_model, compute_share_model, get_tier, convert_size
from cluster_simulator.phase import DelayPhase, ComputePhase, IOPhase
from cluster_simulator.application import Application
from cluster_simulator.analytics import display_run
from cluster_simulator.ephemeral_placement import ClusterBlackBox


c:\Users\a770398\AppData\Local\Continuum\anaconda3\envs\iosea-venv\lib\site-packages\cma\s.py:13: UserWarning:

Could not import matplotlib.pyplot, therefore ``cma.plot()`` etc. is not available



Setting the Cluster:
- with one tier level (HDD)
- with an ephemeral tier:
 

In [3]:
# sim env an data
env = simpy.Environment()
data = simpy.Store(env)
# tier perfs
nvram_bandwidth = {'read':  {'seq': 800, 'rand': 800},
                        'write': {'seq': 400, 'rand': 400}}
ssd_bandwidth = {'read':  {'seq': 200, 'rand': 200},
                    'write': {'seq': 100, 'rand': 100}}
hdd_bandwidth = {'read':  {'seq': 80, 'rand': 80},
                    'write': {'seq': 40, 'rand': 40}}

# registering Tiers
hdd_tier = Tier(env, 'HDD', bandwidth=hdd_bandwidth, capacity=1e12)
ssd_tier = Tier(env, 'SSD', bandwidth=ssd_bandwidth, capacity=200e9)
nvram_tier = Tier(env, 'NVRAM', bandwidth=nvram_bandwidth,
                        capacity=10e9)
# registering Ephemeral Tier
bb = EphemeralTier(env, name="BB", persistent_tier=hdd_tier,
                        bandwidth=nvram_bandwidth, capacity=10e9)

# Define the cluster with 1 persistent and 1 ephemeral
cluster = Cluster(env, tiers=[hdd_tier], ephemeral_tier=bb)

Behavior for no BB use

In [4]:
# logger
#env = simpy.Environment()
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
read = [1e9, 0]
compute = [0,  10]
write = [0, 5e9]
# placement
placement = [0, 0]
use_bb = [False, False]
# simulate the app execution
app1 = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app1.run(cluster, placement=placement, use_bb=use_bb))
env.run()
print(f"application duration = {app1.get_fitness()}")
fig = display_run(data, cluster, width=800, height=900)
fig.show()


application duration = 147.5


Selective SBB per dataset

In [5]:
# logger
# sim env an data

data = simpy.Store(env)
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
compute = [0,  10]
read = [1e9, 0]
write = [0, 5e9]
placement = [0, 0]
use_bb = [False, True] # enabling ephemeral tier for second dataset
app = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app.run(cluster, placement=placement, use_bb=use_bb))
env.run()
print(f"application duration = {app.get_fitness()}")
fig = display_run(data, cluster, width=800, height=900)
fig.show()


application duration = 47.5


What about prefetch ? (buffering read datasets)

In [6]:
# logger
# sim env an data
data = simpy.Store(env)
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
compute = [0,  10]
read = [1e9, 0]
write = [0, 5e9]
placement = [0, 0]
use_bb = [True, False]
app = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app.run(cluster, placement=placement, use_bb=use_bb))
env.run()
print(f"application duration = {app.get_fitness()}")
fig = display_run(data, cluster, width=800, height=900)
fig.show()

application duration = 136.25


Burst Buffer Saturation
 - will choose bb_size < 5GB (max used size)
 

In [7]:
# registering Ephemeral Tier
bb = EphemeralTier(env, name="BB", persistent_tier=hdd_tier,
                        bandwidth=nvram_bandwidth, capacity=4.5e9)

# Define the cluster with 1 persistent and 1 ephemeral
cluster = Cluster(env, tiers=[hdd_tier], ephemeral_tier=bb)
# logger
# sim env an data
data = simpy.Store(env)
logger.remove()
# Simple app: read 1GB -> compute 10s -> write 5GB
compute = [0,  10]
read = [1e9, 0]
write = [0, 5e9]
placement = [0, 0]
use_bb = [False, True] # enabling ephemeral tier for second dataset
app = Application(env, compute=compute, read=read, write=write, data=data)
env.process(app.run(cluster, placement=placement, use_bb=use_bb))
env.run()
print(f"application duration = {app.get_fitness()}")
fig = display_run(data, cluster, width=800, height=900)
fig.show()

application duration = 47.5


- eviction occurs
- buffer saturation should be avoided:
  - only newly copied data to lower tier diminishes the amount of dirty data
  - we can only evict clean data
  - in terms of simulations, it shorten the time interval and the loop become extremly slow (~inf)
- the focus is not on studying what happens in saturation mode, but to size relevantly the BB

Optimizing BB Size (flavor)


In [ ]:
"""
app1 = Application(self.env,
                        compute=[0, 10],
                        read=[1e9, 0],
                        write=[0, 5e9],
                        data=self.data)
app2 = Application(self.env,
                        compute=[0, 20, 30],
                        read=[3e9, 0, 0],
                        write=[0, 5e9, 10e9],
                        data=self.data)
app3 = Application(self.env,
                        compute=[0, 10],
                        read=[4e9, 0],
                        write=[0, 7e9],
                        data=self.data)
                        
apps = [app1, app2, app3]

bb = EphemeralTier(self.env, name="BB", persistent_tier=self.ssd_tier,
        bandwidth=self.nvram_bandwidth, capacity=50e9)
        #capacity=self.get_max_io_volume())
cluster = Cluster(self.env, compute_nodes=2, cores_per_node=5,
        tiers=[self.hdd_tier, self.ssd_tier],
        ephemeral_tier=self.bb)
"""

In [8]:
import simpy
from loguru import logger
import time
import numpy as np
import pandas as pd
import time, os
from itertools import chain
from cluster_simulator.cluster import Cluster, Tier, EphemeralTier, bandwidth_share_model, compute_share_model, get_tier, convert_size
from cluster_simulator.phase import DelayPhase, ComputePhase, IOPhase
from cluster_simulator.application import Application
from cluster_simulator.analytics import display_run
# imports for surrogate models
from sklearn.gaussian_process import GaussianProcessRegressor
from bbo.optimizer import BBOptimizer
# from bbo.optimizer import timeit
from bbo.heuristics.surrogate_models.next_parameter_strategies import expected_improvement

# imports for genetic algorithms
from bbo.heuristics.genetic_algorithm.selections import tournament_pick
from bbo.heuristics.genetic_algorithm.crossover import double_point_crossover
from bbo.heuristics.genetic_algorithm.mutations import mutate_chromosome_to_neighbor
from loguru import logger
from cluster_simulator.ephemeral_placement import ClusterBlackBox
logger.remove()
cbb = ClusterBlackBox()
PARAMETER_SPACE = cbb.parameter_space
# combinations are self.n_tiers ** sum(self.ios) + 2**sum(self.ios)
NBR_ITERATION = 20  # cbb.n_tiers ** sum(cbb.ios)

np.random.seed(5)
bbopt = BBOptimizer(black_box=cbb,
                    heuristic="surrogate_model",
                    max_iteration=NBR_ITERATION,
                    initial_sample_size=60,
                    parameter_space=PARAMETER_SPACE,
                    next_parameter_strategy=expected_improvement,
                    regression_model=GaussianProcessRegressor)
start_time = time.time()
bbopt.optimize()
print("-----------------")
print(f"Total number of iterations : {NBR_ITERATION}")
print(f"{(time.time() - start_time)} seconds spent for finding solution")
print("-----------------")
bbopt.summarize()
print(f"Fitness history : {bbopt.history['fitness']}")
#cbb.save_experiment(filename = "flavor_optim", save=True)

Full BBO param array = [0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1.]
    | app#G9 : tier placement: [0. 1.] | use_bb = [True, True]
    | app#G6 : tier placement: [0. 0. 1.] | use_bb = [False, True, False]
    | app#Y3 : tier placement: [0. 1.] | use_bb = [True, False]
    | runtime = 293.80952380952385 |  BB_size = 15.0 GB
Full BBO param array = [1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
    | app#N6 : tier placement: [1. 0.] | use_bb = [False, False]
    | app#X5 : tier placement: [1. 1. 0.] | use_bb = [True, False, True]
    | app#S2 : tier placement: [1. 0.] | use_bb = [False, True]
    | runtime = 206.62493924629848 |  BB_size = 20.0 GB
Full BBO param array = [1 0 1 1 0 0 0 1 0 0 1 0 1 0]
    | app#I6 : tier placement: [1 0] | use_bb = [False, False]
    | app#M6 : tier placement: [1 1 0] | use_bb = [True, False, True]
    | app#A7 : tier placement: [1 0] | use_bb = [False, True]
    | runtime = 206.62493924629848 |  BB_size = 20.0 GB
-----------------
Total number of iterations : 

Testing naive optimal placement

In [10]:
cbb.display_placement([1]*14)

Displaying result for placement parameter = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    | app#1 : tier placement: [1, 1] | use_bb = [True, True]
    | app#2 : tier placement: [1, 1, 1] | use_bb = [True, True, True]
    | app#3 : tier placement: [1, 1] | use_bb = [True, True]
    | runtime = 193.6296187631139 |  BB_size = 35.0 GB


In [9]:
import numpy as np
import pandas as pd
import os

pathfile = os.path.join(os.getcwd(), "flavor_optim")
df = pd.read_pickle(pathfile)
#sorted_df = df.sort_values(by=['Fitness'], inplace=False, ascending=True)
#sorted_df.head(5)
df.head(3)
#list(df.columns)


,Param,App#W9 tier place,App#W9 use bb,App#K2 tier place,App#K2 use bb,App#W6 tier place,App#W6 use bb,Fitness,BB_size
0,"[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","[0.0, 1.0]","[True, True]","[0.0, 0.0, 1.0]","[False, True, False]","[0.0, 1.0]","[True, False]",293.809524,1.500000e+10
1,"[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0]","[False, False]","[1.0, 1.0, 0.0]","[True, False, True]","[1.0, 0.0]","[False, True]",206.624939,2.000000e+10
2,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0]","[False, True]","[0.0, 1.0, 0.0]","[True, True, True]","[1.0, 0.0]","[True, True]",203.504327,3.400000e+10


Plotting Workflow duration Vs BB Size : do you recognize the plot?


In [11]:
import plotly.express as px
import plotly.graph_objects as go
colors = px.colors.qualitative.Plotly
fig = go.Figure()
fig.add_traces(go.Scatter(x=df['BB_size'], y = df['Fitness'], 
                          text = ["trial_index=" + str(i) for i in list(df.index)],
                          mode = 'markers', line=dict(color=colors[0])))
fig.update_xaxes(title_text='Size of BB in bytes (B = 1e9 = GB)')
fig.update_yaxes(title_text='Workflow duration in seconds')
fig.show()

- each point is a placement configuration:
  - for each dataset:
    - the tier where it should be placed
    - the use or not of the BB to prefetch/buffer 

Show some samples

In [12]:
# indicate the number of trial
cbb.display_placement(df.loc[129]["Param"])

Displaying result for placement parameter = [1 0 1 1 1 1 1 0 0 0 0 1 1 1]
    | app#1 : tier placement: [1 0] | use_bb = [False, False]
    | app#2 : tier placement: [1 1 1] | use_bb = [False, True, True]
    | app#3 : tier placement: [1 1] | use_bb = [True, True]
    | runtime = 177.11271737970768 |  BB_size = 26.0 GB


Efficient frontier / Pareto frontier

In [13]:
from cluster_simulator.utils import convex_hull
points = []
for (xi, yi) in zip(df['BB_size'].tolist(), (df['Fitness']).tolist()):
    points.append((xi,yi))
lower = convex_hull(points)
fig.add_traces(go.Scatter(x=np.array([low[0] for low in lower]),
                            y=np.array([low[1] for low in lower])))
fig.show()

Recommandation are elements from efficient/pareto frontier

In [14]:
rec_trials = [29, 31, 4, 6, 129]
df.loc[rec_trials][df.columns[1:]]

,App#W9 tier place,App#W9 use bb,App#K2 tier place,App#K2 use bb,App#W6 tier place,App#W6 use bb,Fitness,BB_size
29,"[1.0, 1.0]","[False, False]","[0.0, 1.0, 1.0]","[False, False, False]","[1.0, 1.0]","[False, False]",398.214286,0.000000e+00
31,"[0.0, 0.0]","[True, False]","[1.0, 1.0, 1.0]","[False, False, False]","[1.0, 1.0]","[False, False]",363.333333,1.000000e+09
4,"[0.0, 0.0]","[True, True]","[0.0, 0.0, 1.0]","[True, False, False]","[0.0, 1.0]","[False, False]",290.000000,9.000000e+09
6,"[1.0, 0.0]","[True, False]","[1.0, 1.0, 1.0]","[False, False, True]","[1.0, 1.0]","[False, True]",205.342888,1.800000e+10
129,"[1, 0]","[False, False]","[1, 1, 1]","[False, True, True]","[1, 1]","[True, True]",177.112717,2.600000e+10


Display individual points (placement recommandation)

In [ ]:
# if optimization object were kept:cbb.display_placement(placement=bbopt.best_parameters_in_grid)
cbb.display_placement(df.loc[129]["Param"])
#fig1.show()